In [49]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import sparse
from sklearn.preprocessing import normalize

In [22]:
titles = pd.read_csv("lastfm_artist_list.csv")
user_scrobbles = pd.read_csv("lastfm_user_scrobbles.csv")

In [23]:
titles.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 17493 entries, 0 to 17492
Data columns (total 2 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   artist_id    17493 non-null  int64 
 1   artist_name  17493 non-null  object
dtypes: int64(1), object(1)
memory usage: 273.5+ KB


In [24]:
user_scrobbles.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 92792 entries, 0 to 92791
Data columns (total 3 columns):
 #   Column     Non-Null Count  Dtype
---  ------     --------------  -----
 0   user_id    92792 non-null  int64
 1   artist_id  92792 non-null  int64
 2   scrobbles  92792 non-null  int64
dtypes: int64(3)
memory usage: 2.1 MB


In [25]:
user_scrobbles.scrobbles.nunique()

5436

Рассмотрим распределение артистов по пользователям (есть ли в данных пользователи о которых мы слишком мало знаем)

In [19]:
user_scrobbles.groupby('user_id').count().mean()

artist_id    49.044397
scrobbles    49.044397
dtype: float64

In [27]:
artists_name = titles.artist_name.to_dict()

Cоздаем разреженную матрицу, строками которой являются пользователи, а столбцы - артисты.

In [44]:
rows, r_pos = np.unique(user_scrobbles.user_id.values, return_inverse=True)
cols, c_pos = np.unique(user_scrobbles.artist_id.values, return_inverse=True)

In [51]:
iter_sparse = sparse.csr_matrix((user_scrobbles.scrobbles, (r_pos, c_pos)))

In [53]:
iter_sparse

<1892x17493 sparse matrix of type '<class 'numpy.int64'>'
	with 92723 stored elements in Compressed Sparse Row format>

Проведем нормировку значений в матрице и построим матрицу похожести.

In [95]:
Pui = normalize(iter_sparse, norm='l2', axis=1)

In [98]:
sim = Pui.T * Pui

In [117]:
sim.todense()

matrix([[0.00120405, 0.        , 0.        , ..., 0.        , 0.        ,
         0.        ],
        [0.        , 0.00258841, 0.        , ..., 0.        , 0.        ,
         0.        ],
        [0.        , 0.        , 0.00383689, ..., 0.        , 0.        ,
         0.        ],
        ...,
        [0.        , 0.        , 0.        , ..., 0.0228355 , 0.        ,
         0.        ],
        [0.        , 0.        , 0.        , ..., 0.        , 0.00827076,
         0.        ],
        [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
         0.0010429 ]])

In [118]:
[artists_name[i] for i in sim[10734].toarray().argsort()[0][-20:]]

['The Doors',
 'Pink Floyd',
 'Joy Division',
 'Depeche Mode',
 'Marilyn Manson',
 'Paramore',
 'System Of A Down',
 'Arctic Monkeys',
 'Pearl Jam',
 'Green Day',
 'Foo Fighters',
 'Muse',
 'Alice In Chains',
 'Red Hot Chili Peppers',
 'Placebo',
 'The Smashing Pumpkins',
 'Metallica',
 'Radiohead',
 'The Beatles',
 'Nirvana']

Получили item-item рекоммендации.

Теперь user-item рекоммендации.

In [130]:
iter_sparse_T = iter_sparse.T

In [132]:
Piu = normalize(iter_sparse_T, norm='l2', axis=1)

In [199]:
fit = Pui @ Piu @ Pui

In [134]:
fit

<1892x17493 sparse matrix of type '<class 'numpy.float64'>'
	with 18946119 stored elements in Compressed Sparse Row format>

Что слушал пользователь.

In [194]:
init_set = [artists_name[i] for i in np.nonzero(iter_sparse[520])[1]]

Что порекомендовали.

In [200]:
pred_set = [artists_name[i] for i in fit[520].toarray().argsort()[0][-70:]]

In [201]:
set(pred_set).difference(initial_set)

{'30 Seconds To Mars',
 'Britney Spears',
 'Christina Aguilera',
 'Coldplay',
 'David Bowie',
 'Duran Duran',
 'Evanescence',
 'Flyleaf',
 'Him',
 'Iron Maiden',
 'Joy Division',
 'Kings Of Leon',
 'Lady Gaga',
 'Linkin Park',
 'Madonna',
 'Marilyn Manson',
 'Metallica',
 'Muse',
 'Nightwish',
 'Nine Inch Nails',
 'Oasis',
 'Opeth',
 'Pink Floyd',
 'Porcupine Tree',
 'Radiohead',
 'Rammstein',
 'System Of A Down',
 'The Beatles',
 'The Killers',
 'The Smashing Pumpkins',
 'The Smiths',
 'U2'}